In [5]:
from helpers import*
import numpy as np
x_train_input_data, x_train_ids = load_csv_data_x(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 1\dataset\dataset\dataset_to_release\x_train.csv")



In [6]:
print(x_train_input_data.shape)

(328135, 320)


In [7]:
y_train = load_csv_data_y(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 1\dataset\dataset\dataset_to_release\y_train.csv")

In [8]:
def remove_col(x, nan_percentage = 0.8) :

    #Remove columns in which there are too much NaN values (<80%)
    to_delete = []
    for i in range(x.shape[1] - 1, 1, -1):
        num_NaN = np.count_nonzero(np.isnan(x[:,i]))
        p_NaN = num_NaN / x.shape[0]
        if p_NaN > nan_percentage:
            to_delete.append(i)

    x = x[:, [i for i in range(x.shape[1]) if i not in to_delete]]
    return x

In [9]:
x_train_clean1 = remove_col(x_train_input_data)

In [10]:
print(x_train_clean1.shape)

(328135, 204)


In [18]:
def clean_data(data):
    clean_datas = data.copy()

    for i in range(data.shape[1]):
        col = clean_datas[:, i]
        is_nan = np.isnan(col)

        if is_nan.any():
            valid_values = col[~is_nan]
            if valid_values.size > 0:
                median = np.median(valid_values)
                col[is_nan] = median

    return clean_datas

In [19]:
x_train_clean2 = clean_data(x_train_clean1)

In [25]:
print(np.isnan(x_train_clean2).any())

False


In [39]:
def remove_outliers(data):

    filtered_data = data.copy()
    for i in range(filtered_data.shape[1]):
        col_data = filtered_data[:,i]
        q1 = np.percentile(col_data,25)
        q3 = np.percentile(col_data,75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5*iqr
        upper_bound = q3 + 1.5*iqr

        col_data[(col_data<lower_bound)|(col_data>upper_bound)] = np.median(col_data)
        filtered_data[:,i]= col_data
    return filtered_data

In [46]:
x_train_clean3 = remove_outliers(x_train_clean2)

<class 'numpy.ndarray'>


In [45]:
data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [100, 110, 120],
                 [7, 8, 9]])

cleaned_datas = remove_outliers(data)

mean_before = np.mean(data, axis=0)
std_before = np.std(data, axis=0)
mean_after = np.mean(cleaned_datas, axis=0)
std_after = np.std(cleaned_datas, axis=0)
print(data)
print( mean_before)
print(std_before)
print(cleaned_datas)
print( mean_after)
print( std_after)


<class 'numpy.ndarray'>
[[  1   2   3]
 [  4   5   6]
 [100 110 120]
 [  7   8   9]]
[28.   31.25 34.5 ]
[41.62331078 45.51579396 49.40900728]
[[1 2 3]
 [4 5 6]
 [5 6 7]
 [7 8 9]]
[4.25 5.25 6.25]
[2.16506351 2.16506351 2.16506351]


In [56]:
def standardize(data):

    mean = np.mean(data, axis=0)
    std = np.std(data, axis= 0)
    for i in range(len(std)):
        if std[i] < 1e-10:
            std[i] = 1
    standardized_data = (data - mean)/std
    return standardized_data


In [59]:
x_train_clean4 = standardize(x_train_clean3)

In [58]:
data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])

# Appelez la fonction de standardisation sur les données d'exemple
standardized_data = standardize(data)

# Comparez les statistiques des données avant et après
mean_before = np.mean(data, axis=0)
std_before = np.std(data, axis=0)
mean_after = np.mean(standardized_data, axis=0)
std_after = np.std(standardized_data, axis=0)
print("Statistiques des données avant la standardisation:")
print("Moyenne avant :", mean_before)
print("Écart type avant :", std_before)

print("\nStatistiques des données après la standardisation:")
print("Moyenne après :", mean_after)
print("Écart type après :", std_after)

# Vérifiez visuellement les données standardisées
print(data)
print("Données standardisées :")
print(standardized_data)

Statistiques des données avant la standardisation:
Moyenne avant : [4. 5. 6.]
Écart type avant : [2.44948974 2.44948974 2.44948974]

Statistiques des données après la standardisation:
Moyenne après : [0. 0. 0.]
Écart type après : [1. 1. 1.]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
Données standardisées :
[[-1.22474487 -1.22474487 -1.22474487]
 [ 0.          0.          0.        ]
 [ 1.22474487  1.22474487  1.22474487]]


In [128]:
def remove_correlated_columns(data, correlation_threshold=0.1):
    uncorrelated_data = data.copy()
    upper_triangle = np.triu(np.ones((data.shape[1], data.shape[1]), dtype=bool), k=1)

    indices_to_delete = []

    for i in range(data.shape[1]):
        for j in range(i + 1, data.shape[1]):
            if upper_triangle[i, j]:
                corr = np.corrcoef(uncorrelated_data[:, i], uncorrelated_data[:, j])[0, 1]
                if np.abs(corr) >= correlation_threshold:
                    indices_to_delete.append(j)
    
    uncorrelated_data = np.delete(uncorrelated_data, indices_to_delete, axis=1)

    return uncorrelated_data

In [ ]:
x_train_clean5 = remove_correlated_columns(x_train_clean4)

In [129]:
data = np.array([[1, 2, 3, 5],
                 [4, 5, 6, 12],
                 [7, 8, 9, 3.5 ],
                 [10, 11, 12, 54]])

datas_cleaned = remove_correlated_columns(data)
print(data)
print(datas_cleaned)

[[ 1.   2.   3.   5. ]
 [ 4.   5.   6.  12. ]
 [ 7.   8.   9.   3.5]
 [10.  11.  12.  54. ]]
[[ 1.]
 [ 4.]
 [ 7.]
 [10.]]


In [ ]:
import pandas as pd 
x_train = pd.read_csv(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 1\dataset\x_train.csv")
y_train = pd.read_csv(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 1\dataset\y_train.csv")
x_test = pd.read_csv(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 1\dataset\x_test.csv")

In [ ]:
x_train.describe()

In [ ]:
x_train.PREGNANT


In [ ]:
x_test_y_b, x_test_input_data, x_test_ids = load_csv_data(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 1\dataset\x_test.csv")

In [ ]:
x_train.head()